# K2496 example

Quantify K2496, a Ba, Ti, Si and O containing engineered glass.

In [ ]:
# Load DrWatson project manager and enable custom environment
using DrWatson
@quickactivate("HyperspectraWithNeXL")
# Load the necessary NeXL libraries
using NeXLSpectrum

# Load 3rd party libraries for plotting and tabulation
using Gadfly, DataFrames

path = joinpath(datadir(),"exp_raw","K2496")
# Read the spectra from disk
k2496 = loadspectrum.(joinpath(path,"K2496_$(i).msa") for i in 1:3)
# Plot them using a method of Gadfly.plot(...) specialized for Spectrum items
set_default_plot_size(8inch,3inch)
plot(k2496..., klms=[ n"O", n"Si", n"Ti", n"Ba" ], xmax=10.0e3)

Define a BasicEDS detector object with the properties of the measurement device.  Then apply the detector to the measured spectra.

In [ ]:
det = BasicEDS(4096, -480.40409, 5.00525, 132.0, 110, 
         Dict(KShell=>n"B", LShell=>n"Ca", MShell=>n"Cs"))
k2496 = map(s->apply(s,det), k2496)
det

Compute the filtered reference spectra from the standards.  Sanbornite acts as a reference for both Si and Ba.  But Ba requires a second spectrum ("BaCl2 std.msa") because the O in Sanbornite has an interference with the Ba M-family peaks.

In [ ]:
refs = references([
  reference(n"Si", joinpath(path,"Sanbornite std.msa"), mat"BaSi2O5"),
  reference(n"Ba", joinpath(path,"Sanbornite std.msa"), mat"BaSi2O5"),
  reference(n"Ba", joinpath(path,"BaCl2 std.msa"), mat"BaCl2"),
  reference(n"O", joinpath(path,"MgO std.msa"), mat"MgO"),
  reference(n"Ti", joinpath(path,"Ti std.msa"), mat"Ti") ], det)
ENV["columns"]=400
asa(DataFrame, refs)

Fit (fit_spectra(...)) and then matrix correct (quantify(...)) the unknown spectra.

In [ ]:
fs=fit_spectrum(k2496, refs)
q=quantify.(fs)

fs is a vector of FilterFitResult items.  Plotting a FilterFitResult shows the raw spectral data (blue), the residual spectrum (red) and the rois and k-ratios in grey above the peaks.

In [ ]:
plot(fs[1])

In [ ]:
asa(DataFrame, fs[1])

The plot may be saved in various formats (including SVG, PNG and PDF).

In [ ]:
# using Cairo, Fontconfig
plot(fs[1]) |> SVG(joinpath(plotsdir(), "K2496 residual.svg"), 8inch, 3inch)
# plot(fs[1]) |> PNG(joinpath(plotsdir(), "K2496 residual.png"), 8inch, 3inch)
# plot(fs[1]) |> PDF(joinpath(plotsdir(), "K2496 residual.pdf"), 8inch, 3inch)

Output the quant results.  Write the results to a CSV file.

In [ ]:
df = asa(DataFrame, q, nominal=parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal"))
using CSV
CSV.write(joinpath(datadir(),"exp_pro","K2496 quant.csv"), df)
df

In [ ]:
describe(asa(DataFrame, q)[:,2:end], :mean, :std)

Tabulating a single IterationResult item provides extensive analysis details including uncertainties and matrix correction factors.

In [ ]:
asa(DataFrame, q[1])

In [ ]:
using PrettyTables
tbl = vcat( (asa(DataFrame,qi) for qi in q)...) 
mkpath(joinpath(papersdir(),"Tables"))
open(joinpath(papersdir(),"Tables","K2496 results.tex"),"w") do io
  pretty_table(io, tbl, nosubheader=true, backend=:latex, 
            label="Exhaustive quantitative analysis results from 3 measurements of K2496 glass.")
end
tbl

QED